# Parte 2. Feature Selection
Usaremos el metodo de ramdonforest para la selección de características.

In [1]:
import os
import sys

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.insert(0, project_root)

os.chdir(project_root)
print(f"Directorio de trabajo actual cambiado a: {os.getcwd()}")

Directorio de trabajo actual cambiado a: /Users/david.bayter/Documents/Private/challenge-ds-meli


In [2]:
import pandas as pd
import numpy as np
from src.modeling.feature_selection import feature_selection_random_forest
from sklearn.preprocessing import LabelEncoder
from config.config import configs
from src.utils.convert_datatype_utils import set_datatypes

In [3]:
df = pd.read_csv('data/processed/df_to_modeling.csv')
df.dtypes

condition                             int64
base_price                          float64
seller_id                             int64
listing_type_id                      object
price                               float64
buying_mode                          object
category_id                          object
international_delivery_mode          object
accepts_mercadopago                    bool
currency_id                          object
automatic_relist                       bool
status                               object
initial_quantity                      int64
sold_quantity                         int64
available_quantity                    int64
seller_address_state.name            object
seller_address_city.name             object
sub_status_deleted                     bool
sub_status_expired                     bool
sub_status_suspended                   bool
shipping_local_pick_up                 bool
shipping_free_shipping                 bool
shipping_mode                   

In [5]:
df = set_datatypes(df, numericas=configs.col_numericas, categoricas_nom=configs.col_categoricas, booleanas=configs.col_booleanas)
df.dtypes

Columnas numéricas convertidas.
Columnas booleanas convertidas.
Error: La columna 'site_id' no se encontró en el DataFrame.
Columnas categóricas convertidas.


condition                              int64
base_price                           float64
seller_id                           category
listing_type_id                     category
price                                float64
buying_mode                         category
category_id                         category
international_delivery_mode         category
accepts_mercadopago                     bool
currency_id                         category
automatic_relist                        bool
status                              category
initial_quantity                       int64
sold_quantity                          int64
available_quantity                     int64
seller_address_state.name           category
seller_address_city.name            category
sub_status_deleted                      bool
sub_status_expired                      bool
sub_status_suspended                    bool
shipping_local_pick_up                  bool
shipping_free_shipping                  bool
shipping_m

In [13]:
selected_features, importance_df, rf_model = feature_selection_random_forest(
        df_label_encoded,
        target_col='condition',
        n_features=15,
        n_estimators=600,
        max_depth=15
    )

Iniciando la selección de características con One-Hot Encoding...
El número de características después de la codificación es: 71
Entrenando el modelo Random Forest...

Accuracy en entrenamiento: 0.8831
Accuracy en test: 0.8614

Top 15 features seleccionadas:
                          feature  importance
33           listing_type_id_free    0.202333
6                initial_quantity    0.185549
8              available_quantity    0.167715
7                   sold_quantity    0.060708
2                           price    0.060432
0                      base_price    0.058747
3                     category_id    0.034328
32               title_cont_nuevo    0.021616
9        seller_address_city.name    0.018423
1                       seller_id    0.017884
29                 warranty_group    0.017231
23  payment_group_Tarjeta_Credito    0.014158
38         listing_type_id_silver    0.013410
31               title_cont_usado    0.013236
5                automatic_relist    0.009986


In [ ]:
import os
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from xgboost import XGBClassifier
from xgboost import XGBRegressor
from sklearn.metrics import cohen_kappa_score, precision_score
from sklearn.metrics import matthews_corrcoef, mean_squared_error, log_loss
from sklearn.metrics import f1_score, recall_score, roc_auc_score

dfs['condition'] = dfs['condition'].replace('new', 0)
dfs['condition'] = dfs['condition'].replace('used', 1)

scaled_features = dfs.copy()
  
col_names = ['warranty', 'initial_quantity', 'available_quantity', 'sold_quantity',
       'base_price', 'price', 'Giro_postal', 'free_shipping', 'local_pick_up',
       'Contra_reembolso', 'Acordar_con_el_comprador', 'Cheque_certificado',
       'Efectivo', 'Transferencia_bancaria', 'Aceptan_Tarjeta',
       'automatic_relist', 'accepts_mercadopago', 'days_active']

features = scaled_features[col_names]
scaler = StandardScaler().fit(features.values)
features = scaler.transform(features.values)
scaled_features[col_names] = features


X = scaled_features.drop(columns=['condition'], axis=1)
#X = dfs.drop(columns='condition')
y = scaled_features.condition


X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state=7)
Y_train = Y_train
Y_test = Y_test

full_pipeline = ColumnTransformer([('cat', OneHotEncoder(handle_unknown='ignore'), X_train.columns)], remainder='passthrough')

encoder = full_pipeline.fit(X_train)
X_train_enc = encoder.transform(X_train)
X_test_enc = encoder.transform(X_test)

# train the model
model = xgb.XGBClassifier(n_estimators= 200,
                             max_depth= 30,                         # Lower ratios avoid over-fitting. Default is 6.
                             objective = 'binary:logistic',         # Default is reg:squarederror. 'multi:softprob' for multiclass and get proba.  
                             #num_class = 2,                        # Use if softprob is set.
                             reg_lambda = 10,                       # Larger ratios avoid over-fitting. Default is 1.
                             gamma = 0.3,                           # Larger values avoid over-fitting. Default is 0. # Values from 0.3 to 0.8 if you have many columns (especially if you did one-hot encoding), or 0.8 to 1 if you only have a few columns.
                             alpha = 1,                             # Larger ratios avoid over-fitting. Default is 0.
                             learning_rate= 0.10,                   # Lower ratios avoid over-fitting. Default is 3.
                             colsample_bytree= 0.7,                 # Lower ratios avoid over-fitting.
                             scale_pos_weight = 1,                  # Default is 1. Control balance of positive and negative weights, for unbalanced classes.
                             subsample = 0.1,                       # Lower ratios avoid over-fitting. Default 1. 0.5 recommended. # 0.1 if using GPU.
                             min_child_weight = 3,                  # Larger ratios avoid over-fitting. Default is 1.
                             missing = np.nan,                      # Deal with missing values
                             num_parallel_tree = 2,                 # Parallel trees constructed during each iteration. Default is 1.
                             importance_type = 'weight',
                             eval_metric = 'auc',
                             #use_label_encoder = True,
                             #enable_categorical = True,
                             verbosity = 1,
                             nthread = -1,                          # Set -1 to use all threads.
                             #use_rmm = True,                       # Use GPU if available
                             tree_method = 'auto', # auto           # 'gpu_hist'. Default is auto: analyze the data and chooses the fastest.
                             #gradient_based = True,                # If True you can set subsample as low as 0.1. Only use with gpu_hist 
                            )

# fit model              
model.fit(X_train_enc, Y_train.values.ravel(),
          # early_stopping_rounds=20
         )

# check best ntree limit
display(model.best_ntree_limit)

# extract the training set predictions
preds_train = model.predict(X_train_enc,
                            ntree_limit=model.best_ntree_limit
                           )
# extract the test set predictions
preds_test = model.predict(X_test_enc,
                           ntree_limit=model.best_ntree_limit
                           )